# Tsunami Vortex Analysis: Self-Contained Execution Notebook

This notebook is the execution-facing guide for the current repository architecture.
It is aligned with:

- Driver: `Scripts/Drivers/Tsunami_Vorticity_Emulator.m`
- Dispatcher path: `Scripts/Infrastructure/Runners/ModeDispatcher.m`
- Method runner path: `Scripts/Solvers/run_simulation_with_method.m`
- Output roots: `Results/` (runtime) and `Artifacts/` (tests/notebook artifacts)

## Scope and contract
- This notebook is self-contained for **currently executable methods**.
- Method execution uses `run_simulation_with_method` for method-level runs.
- Dispatcher notes are included to clarify current mode compatibility.


## Method capability snapshot (current codebase)

| Area | Finite Difference | Spectral | Finite Volume |
|---|---|---|---|
| `run_simulation_with_method` | Runnable | Runnable | Runnable |
| `ModeDispatcher` evolution/convergence/sweep routing | Runnable | Not yet in dispatcher callbacks | Not yet in dispatcher callbacks |

### Important distinction
- `run_simulation_with_method` supports method-level runs directly.
- `ModeDispatcher` currently enforces stricter mode support (FD-first path).


In [ ]:
from __future__ import annotations

import subprocess
from pathlib import Path
from textwrap import dedent

REPO_ROOT = Path.cwd().resolve()
ARTIFACT_ROOT = REPO_ROOT / "Artifacts" / "notebook_runs"
ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Notebook artifacts: {ARTIFACT_ROOT}")


## MATLAB bridge utility

The helper below launches MATLAB in batch mode, runs one method with small settings,
and stores a `.mat` artifact under `Artifacts/notebook_runs/`.


In [ ]:
def run_method_smoke(method_key: str, nx: int = 32, ny: int = 32, dt: float = 0.01, tfinal: float = 0.1) -> subprocess.CompletedProcess:
    method_to_analysis = {
        "finite_difference": "Finite Difference",
        "spectral": "Spectral",
        "finite_volume": "Finite Volume",
    }
    if method_key not in method_to_analysis:
        raise ValueError(f"Unsupported method_key: {method_key}")

    out_file = ARTIFACT_ROOT / f"analysis_{method_key}.mat"

    matlab_script = dedent(f"""
    try
        repo_root = '{REPO_ROOT.as_posix()}';
        addpath(genpath(fullfile(repo_root, 'Scripts')));
        addpath(fullfile(repo_root, 'utilities'));

        params = Parameters();
        params.method = '{method_key}';
        params.analysis_method = '{method_to_analysis[method_key]}';
        params.Nx = {nx};
        params.Ny = {ny};
        params.dt = {dt};
        params.Tfinal = {tfinal};
        params.num_plot_snapshots = 5;
        params.num_snapshots = 5;
        params.snap_times = linspace(0, params.Tfinal, params.num_snapshots);
        params.create_animations = false;

        [~, analysis] = run_simulation_with_method(params);
        save('{out_file.as_posix()}', 'analysis', '-v7.3');
    catch ME
        disp(getReport(ME, 'extended'));
        exit(1);
    end
    """)

    result = subprocess.run(
        ["matlab", "-batch", matlab_script],
        capture_output=True,
        text=True,
    )
    return result


## Execute smoke runs for currently executable methods

These are intentionally small runs for notebook reproducibility.


In [ ]:
methods = ["finite_difference", "spectral", "finite_volume"]
results = {}

for m in methods:
    proc = run_method_smoke(m)
    results[m] = {
        "returncode": proc.returncode,
        "stdout_tail": "\n".join(proc.stdout.strip().splitlines()[-12:]),
        "stderr_tail": "\n".join(proc.stderr.strip().splitlines()[-12:]),
    }
    print(f"[{m}] return code = {proc.returncode}")

results


In [ ]:
generated = sorted(ARTIFACT_ROOT.glob("analysis_*.mat"))
for p in generated:
    print(p)


## Dispatcher compatibility note

For mode-oriented studies (`Evolution`, `Convergence`, `ParameterSweep`, `Plotting`),
`ModeDispatcher` is the canonical route. At this stage, it is intentionally stricter than
`run_simulation_with_method` and should be treated as the authoritative mode compatibility layer.


## Mathematical model alignment (vorticity-streamfunction)

Core formulation used in this repository:

1. Vorticity transport: $\partial_t \omega + J(\psi,\omega) = \nu \nabla^2 \omega$
2. Streamfunction inversion: $\nabla^2 \psi = -\omega$
3. Velocity recovery: $u = -\partial_y\psi$, $v = \partial_x\psi$

Finite-difference implementation aligns with the conservative Arakawa-style discretization
in `Scripts/Methods/FiniteDifference/FiniteDifferenceMethod.m`.


## Computational efficiency remarks

- FD path uses conservative Jacobian handling and sparse Poisson workflows where configured.
- Small snapshot counts keep artifact size bounded for exploratory runs.
- Animation generation is decoupled from plot snapshots (`num_plot_snapshots` vs animation frames).
- This notebook uses smoke-scale grids; production studies should increase resolution and add
  convergence/sensitivity validation.


## Innovation timeline (from repository history)

Selected milestones reflected in current architecture:

- Method/mode architecture refactor and stricter dispatcher contracts.
- Structured pathing via `PathBuilder` and run identifiers via `RunIDGenerator`.
- Sustainability tooling foundation (`HardwareMonitorBridge`, `EnergySustainabilityAnalyzer`).
- Documentation and verification passes with architecture compliance tests.

Use `git log --oneline` in the repo root for the full chronological record.


## Next experiments from this notebook

1. Increase `Nx`, `Ny` and compare runtime/quality tradeoffs by method.
2. Integrate canonical report payload export per run.
3. Add MP4 media checks and embed outputs for notebook-native playback review.
